In [ ]:
#Script to find Table 3 and S4 DFT values

import os
import re
import pandas as pd
import numpy as np

# Full paths to DFT directories (fhi-aims_data)
dft_paths = {
    '1H': r"pyroxene_6/1_Hydrogen/final_fhi-aims_data/",
    '0H': r"data_no_hydrogen/P6/fhi-aims_outputs/",
    '3H': r"pyroxene_6/3_Hydrogen/final_fhi-aims_data/"
}

def extract_dft_cpu_time(file_path):
    lines = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()

    # Find last occurrence of "Detailed time accounting"
    for i in range(len(lines)-1, 0, -1):
        if "Detailed time accounting" in lines[i]:
            # The next line is the one we want
            if i+1 < len(lines):
                match = re.search(r'Total time\s*:\s*([\d.]+)\s*s', lines[i+1])
                if match:
                    return float(match.group(1))
            break  # stop searching after last block
    return None


def process_dft_folder(base_path):
    times = []
    for i in range(1, 51):
        opt_file = os.path.join(base_path, str(i), "optimization.out")
        if os.path.exists(opt_file):
            cpu_time = extract_dft_cpu_time(opt_file)
            print(f"{base_path}: {cpu_time:.2f} h")
            if cpu_time is not None:
                times.append(cpu_time)
    return times

# Collect results
results = []

for name, path in dft_paths.items():
    times = process_dft_folder(path)
    if times:
        stats = {
            'Molecule': name,
            'N Files': len(times),
            'Total Time (h)': sum(times) / 3600,
            'Average (h)': np.mean(times) / 3600,
            'Max (h)': np.max(times) / 3600,
            'Min (h)': np.min(times) / 3600,
            'Std Dev (h)': np.std(times) / 3600,
        }
        results.append(stats)
    else:
        print(f"No valid optimization.out files found for {name}")

# Export
df = pd.DataFrame(results)
df.to_excel("DFT_CPU_Stats_P6.xlsx", index=False)
print("Exported DFT stats to DFT_CPU_Stats_P6.xlsx")


In [ ]:
#Script to find Table 3 and S4 TDDFT values

import os
import re
import pandas as pd
import numpy as np

# Full paths to DFT directories (fhi-aims_data)
dft_paths = {
    '1H': r"pyroxene_6/1_Hydrogen/final_fhi-aims_data/",
    '0H': r"data_no_hydrogen/P6/fhi-aims_outputs/",
    '3H': r"pyroxene_6/3_Hydrogen/final_fhi-aims_data/"
}

def parse_cpu_time(line):
    match = re.search(r'(\d+)\s+days\s+(\d+)\s+hours\s+(\d+)\s+minutes\s+([\d.]+)\s+seconds', line)
    if match:
        days, hours, minutes, seconds = map(float, match.groups())
        return days * 86400 + hours * 3600 + minutes * 60 + seconds
    return None

def process_folder(path):
    times = []
    for i in range(1, 51):  # out_1.log to out_50.log
        file = os.path.join(path, f"out_{i}.log")
        if os.path.exists(file):
            with open(file, 'r', encoding='utf-8', errors='ignore') as f:
                for line in f:
                    if "Job cpu time:" in line:
                        t = parse_cpu_time(line)
                        if t is not None:
                            times.append(t)
                        break
    return times

# Collect data
results = []

for name, path in data_paths.items():
    times = process_folder(path)
    if times:
        stats = {
            'Molecule': name,
            'N Files': len(times),
            'Total Time (h)': sum(times) / 3600,
            'Average (h)': np.mean(times) / 3600,
            'Max (h)': np.max(times) / 3600,
            'Min (h)': np.min(times) / 3600,
            'Std Dev (h)': np.std(times) / 3600,
        }
        results.append(stats)
    else:
        print(f"No logs found in {path}")

# Create DataFrame and export to Excel
df = pd.DataFrame(results)
output_file = "TDDFT_CPU_Stats_P6.xlsx"
df.to_excel(output_file, index=False)
print(f"Exported summary to {output_file}")



In [ ]:
#Script to extract Figures 33 and S9


import numpy as np
import matplotlib.pyplot as plt
import glob, re
from pathlib import Path
from matplotlib import cm

# ====================================================
# Fixed parameters (modify as desired)
# ====================================================
target_gap = 5.7    # target HOMO–LUMO gap (eV)
window = 1          # half‑width (eV)
E_min = target_gap - window
E_max = target_gap + window

# ====================================================
# Define base path for UV files.
# (Assumes UV files are named like "signals_1.txt", "signals_2.txt", … "signals_50.txt".)
# ====================================================
uv_base_path = r"pyroxene_4\final_uv_data"

# Get list of all UV files in the directory.
uv_files = glob.glob(uv_base_path + r"\signals_*.txt")
# Sort the file list numerically by extracting the number from the filename.
uv_files.sort(key=lambda x: int(re.search(r"signals_(\d+)", Path(x).stem).group(1)))
selected_uv_files = uv_files  # Process all files

# ====================================================
# Define the common grid for Gaussian broadening.
# x_total is in nm (as in your original code).
# ====================================================
x_total = np.linspace(1, 400, 10000)

# ====================================================
# Function: Process a single UV file.
# ====================================================
def process_uv_file(file, E_min, E_max):
    """
    Reads a UV file (with two columns: wavelength in nm and intensity),
    applies Gaussian broadening on the grid x_total (in nm) using your formula,
    converts x_total to energy in eV using:
          E (eV) = 1239.84 / (wavelength in nm),
    reverses the arrays so that energy increases,
    and restricts the result to the energy window [E_min, E_max].
    
    Returns:
      E_window: 1D array of energy values (eV) within the window.
      y_window: 1D array of the corresponding UV intensity (un-normalized) within the window.
    """
    data = np.loadtxt(file)
    x_nm = data[:, 0]   # wavelengths in nm
    f = data[:, 1]      # intensities
    y = np.zeros_like(x_total)
    for i in range(len(f)):
        y += (1.3062974e8) * (f[i] / (1e7/6400)) * np.exp(-(((1/x_total) - (1/x_nm[i]))/(1.0/6400))**2)
    # Convert x_total (nm) to energy (eV)
    E_uv = 1239.84 / x_total  
    # Reverse arrays so that energy increases (since energy decreases as wavelength increases)
    E_uv_sorted = E_uv[::-1]
    y_sorted = y[::-1]
    # Restrict to the energy window:
    mask = (E_uv_sorted >= E_min) & (E_uv_sorted <= E_max)
    E_window = E_uv_sorted[mask]
    y_window = y_sorted[mask]
    return E_window, y_window

# ====================================================
# Process all selected UV files and collect data.
# ====================================================
# Create an array for folder numbers using the numeric values extracted from file names.
folder_numbers = []
uv_data_list = []  # Will store each file's UV intensity (1D array over energy)
for uv_file in selected_uv_files:
    # Extract the folder number from the file name (e.g., "signals_23.txt" → 23)
    file_stem = Path(uv_file).stem  # e.g., "signals_23"
    m = re.search(r"signals_(\d+)", file_stem)
    if m:
        folder_num = int(m.group(1))
    else:
        folder_num = 0
    folder_numbers.append(folder_num)
    
    # Process the UV file for the specified energy window.
    E_window, y_window = process_uv_file(uv_file, E_min, E_max)
    uv_data_list.append(y_window)

# Convert the list of UV intensity arrays into a 2D array.
# Each row corresponds to one file.
Z = np.array(uv_data_list)

# Global min–max normalization across all UV data:
global_min = np.min(Z)
global_max = np.max(Z)
Z_norm = (Z - global_min) / (global_max - global_min)

# ====================================================
# Obtain a common energy axis.
# ====================================================
# We assume all files produce the same energy axis; use the first file.
E_common, _ = process_uv_file(selected_uv_files[0], E_min, E_max)

# ====================================================
# Create a meshgrid for the pseudocolor plot.
# x-axis: the extracted folder numbers, y-axis: energy (eV)
# ====================================================
n_folders = len(folder_numbers)
n_energy = Z_norm.shape[1]
X = np.repeat(np.array(folder_numbers).reshape(n_folders, 1), n_energy, axis=1)
Y = np.repeat(np.array(E_common).reshape(1, n_energy), n_folders, axis=0)

# ====================================================
# Plot the pseudocolor graph.
# ====================================================
plt.figure(figsize=(8,6))
plt.pcolormesh(X, Y, Z_norm, cmap=cm.coolwarm, shading='auto')
plt.xlabel("Isomer Number")
plt.ylabel("Energy (eV)")
plt.colorbar(label="Normalized UV Intensity")
plt.title("Pseudocolour of UV Intensity vs. Isomer Number and Energy (eV) (Pyroxene 4 (2H))", fontsize=12)
plt.axhline(y=5.7, color='green', linestyle="--")

# Set x-axis ticks so that every other folder number is labeled.
xticks = np.arange(min(folder_numbers), max(folder_numbers)+1, 2)
plt.xticks(xticks)

plt.savefig("pseudocolour_P4.png", dpi=300)
plt.show()

In [ ]:
#Example script to extract Figure 36

import re
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# Step 1: Define paths to directories and files
optimisation_root_dir = Path(r"pyroxene_4\final_fhi-aims_data")
excel_file_path = r"New Average\UV intensities P4.xlsx"

# Step 2: Get parameters for HOMO-n and LUMO+x from user
n = int(input("Enter the value of n for HOMO-n: "))
x = int(input("Enter the value of x for LUMO+x: "))

# Step 3: Load intensity data from Excel and ensure folder names are strings
df_intensity = pd.read_excel(excel_file_path)
df_intensity["File Number"] = df_intensity["File Number"].astype(str)

# Step 4: Prepare lists to store data for plotting and labelling
file_numbers = []
gap_values = []
intensity_values = []

# Function to parse the eigenvalue block from an optimization.out file content
def parse_eigenvalue_block(content):
    eigenvalue_data = []
    # Find the header (searching from the end of the file)
    state_header_index = next(
        (i for i, line in enumerate(reversed(content)) if "State    Occupation    Eigenvalue [Ha]" in line), None
    )
    if state_header_index is not None:
        state_header_index = len(content) - state_header_index - 1
        eigenvalue_block = content[state_header_index:]
        for line in eigenvalue_block[1:]:  # Skip the header line
            tokens = line.split()
            # Check if the line has 4 tokens and that the first 3 can be parsed as numbers
            if len(tokens) == 4 and all(c.replace('.', '', 1).replace('-', '', 1).isdigit() for c in tokens[:3]):
                try:
                    eigenvalue_data.append([int(tokens[0]), float(tokens[1]), float(tokens[2]), float(tokens[3])])
                except ValueError:
                    continue
        return pd.DataFrame(eigenvalue_data, columns=["State", "Occupation", "Eigenvalue [Ha]", "Eigenvalue [eV]"])
    return None

# Step 5: Extract gap values and match with intensity data for each folder
for opt_folder in optimisation_root_dir.iterdir():
    if opt_folder.is_dir():
        opt_file = opt_folder / "optimization.out"
        folder_name = opt_folder.name  # This will be our label
        
        if opt_file.exists():
            with open(opt_file, 'r') as f:
                content = f.readlines()
                
                eigenvalue_df = parse_eigenvalue_block(content)
                if eigenvalue_df is not None:
                    # Identify HOMO: last state with full occupation (2.00000)
                    homo_row = eigenvalue_df[eigenvalue_df["Occupation"] == 2.00000].iloc[-1]
                    # Identify LUMO: first state with occupation less than 2.00000
                    lumo_row = eigenvalue_df[eigenvalue_df["Occupation"] < 2.00000].iloc[0]
                    
                    homo_index = homo_row["State"]
                    lumo_index = lumo_row["State"]
                    
                    # Calculate energies for HOMO-n and LUMO+x if available
                    homo_n_energy = eigenvalue_df.loc[eigenvalue_df["State"] == homo_index - n, "Eigenvalue [eV]"].values
                    lumo_x_energy = eigenvalue_df.loc[eigenvalue_df["State"] == lumo_index + x, "Eigenvalue [eV]"].values
                    
                    if homo_n_energy.size > 0 and lumo_x_energy.size > 0:
                        gap = lumo_x_energy[0] - homo_n_energy[0]
                        gap_values.append(gap)
                        file_numbers.append(folder_name)
                        
                        # Find matching intensity for the folder from the Excel data
                        if folder_name in df_intensity["File Number"].values:
                            intensity = df_intensity.loc[df_intensity["File Number"] == folder_name, "Peak Intensity (Mb)"].values
                            if intensity.size > 0:
                                intensity_values.append(intensity[0])
                            else:
                                print(f"No matching intensity found for folder {folder_name}")
                        else:
                            print(f"No matching intensity found for folder {folder_name}")
                    else:
                        print(f"Gap calculation failed for folder {folder_name}")
                else:
                    print(f"No eigenvalue block found in folder {folder_name}")
        else:
            print(f"Optimization file not found in folder {folder_name}")

# Step 6: Create a DataFrame for plotting and labelling
df_plot = pd.DataFrame({
    "File Number": file_numbers,
    "Gap Energy": gap_values,
    "Peak Intensity (Mb)": intensity_values
})

# Step 7: Plot the results with labels
plt.figure(figsize=(8, 6))
plt.scatter(df_plot["Gap Energy"], df_plot["Peak Intensity (Mb)"]/4, c="blue", edgecolor="black")
plt.title(f"HOMO-{n} to LUMO+{x} Gap vs Intensity (P4 + 2H)")
plt.xlabel("Gap Energy (eV)")
plt.ylabel(r'$\sigma$ / Si atom [Mb]', fontweight='bold')
plt.grid(True)

# Add annotation for each data point using the file number as the label
for idx, row in df_plot.iterrows():
    plt.annotate(str(row["File Number"]),
                 (row["Gap Energy"], row["Peak Intensity (Mb)"]/4),
                 textcoords="offset points", xytext=(5, 5), fontsize=8)

plt.show()


In [ ]:
#Script to extract peak and average intensity data used in Figures 36-37

import numpy as np
import glob
import re
import pandas as pd
from pathlib import Path

# Base path to your files
base_path = (r"C:/Users/charl/OneDrive - Imperial College London/Chemistry/Y4 Chemistry/Master’s Project/Data_UV_Hydrogenated_Pyroxenes/my_calculations/pyroxene_6/3_Hydrogen/final_uv_data/")

def gaussian_broadening(x_nm, f, x_total, sigma):
    """ Apply Gaussian broadening in the inverse-micron domain. """
    x_file_inv = 1e3 / x_nm   # Convert wavelengths to inverse microns
    y = np.zeros_like(x_total)

    # Apply Gaussian broadening to each peak
    for i in range(len(f)):
        y += f[i] * np.exp(-((x_total - x_file_inv[i]) / sigma) ** 2)

    # Apply scaling factor for correct intensity units
    factor = 1.3062974e8 / (1e7 / 6200)
    y *= factor

    # Convert intensity to Megabarns (Mb)
    y *= 3.82353216 * 10**(-21) * 10**(18)

    return y

def extract_peak_and_avg_intensity(y, x_total, peak_range=(4.08, 4.82), avg_bounds=(4.08, 4.82)):
    """ 
    Extracts peak intensity and calculates average intensity within a fixed range (avg_bounds).
    
    Parameters:
        y          : broadened intensity array
        x_total    : x-axis grid in inverse microns (sorted ascending)
        peak_range : Tuple (lower, upper) for valid peak selection range in μm⁻¹
        avg_bounds : Tuple (lower, upper) for average intensity calculation range
    
    Returns:
        peak_intensity : Maximum intensity within valid peak range (Mb)
        peak_wavenumber: Wavenumber where peak occurs (μm⁻¹)
        avg_intensity  : Average intensity between avg_bounds (Mb)
    """
    # Get peak within the desired range
    peak_low_idx = np.searchsorted(x_total, peak_range[0])
    peak_high_idx = np.searchsorted(x_total, peak_range[1])
    
    if peak_high_idx > peak_low_idx:
        valid_region_intensity = y[peak_low_idx:peak_high_idx]
        valid_region_x = x_total[peak_low_idx:peak_high_idx]
        peak_idx = np.argmax(valid_region_intensity)
        peak_intensity = valid_region_intensity[peak_idx]
        peak_wavenumber = valid_region_x[peak_idx]
    else:
        return np.nan, np.nan, np.nan

    # Now compute average intensity over the fixed window: 4.08–4.82 μm⁻¹
    avg_low_idx = np.searchsorted(x_total, avg_bounds[0])
    avg_high_idx = np.searchsorted(x_total, avg_bounds[1])

    if avg_high_idx > avg_low_idx:
        avg_region_intensity = y[avg_low_idx:avg_high_idx]
        avg_region_x = x_total[avg_low_idx:avg_high_idx]
        avg_intensity = np.trapz(avg_region_intensity, avg_region_x) / (avg_bounds[1] - avg_bounds[0])
    else:
        avg_intensity = np.nan

    return peak_intensity, peak_wavenumber, avg_intensity


# Get all the relevant files and sort them
all_files = glob.glob(base_path + "signals_*.txt")
all_files.sort()

results = []

# Define x_total grid using inverse microns
wavelength_range = np.linspace(175, 250, 1000)  # in nm
x_ev = 1239.84 / wavelength_range
x_ev_corrected = x_ev + 0.1  # Small energy correction
wavelength_corrected = 1239.84 / x_ev_corrected  # Convert back to nm
x_total_unsorted = 1e3 / wavelength_corrected  # Convert to inverse microns
x_total = np.sort(x_total_unsorted)

sigma = 0.081  # Gaussian broadening width in μm⁻¹

# Process each file
for file in all_files:
    data = np.loadtxt(file)
    x_nm = data[:, 0]
    f = data[:, 1]
    
    # Apply Gaussian broadening
    y = gaussian_broadening(x_nm, f, x_total, sigma)
    
    # Extract peak and average intensity **only if peak is within 4.1–4.8 μm⁻¹**
    peak_intensity, peak_wavenumber, avg_intensity = extract_peak_and_avg_intensity(y, x_total, peak_range=(4.08, 4.82))

    if not np.isnan(peak_wavenumber):
        peak_wavenumber_rounded = round(peak_wavenumber, 2)
    else:
        peak_wavenumber_rounded = np.nan

    # Extract the file number from the filename
    file_number = re.search(r"signals_(\d+)\.txt", Path(file).name).group(1)
    results.append((int(file_number), peak_intensity, avg_intensity, peak_wavenumber_rounded))

# Create a DataFrame with the updated column names
df = pd.DataFrame(results, columns=['File Number', 'Peak Intensity (Mb)', 
                                    'Avg Intensity (Mb)', 'Peak Position (Wavenumber, μm⁻¹)'])

# Save the DataFrame to an Excel file
df.to_excel("UV intensities P4.xlsx", index=False)
print("Extracted data written to UV intensities P4 .xlsx")


In [ ]:
#Script to extract Figure 37

import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Load extracted Mg-H distances from CSV
distances_file = "mg_h_distances_P4.csv"  # Ensure this file is generated from the previous script
df_distances = pd.read_csv(distances_file)

# Step 2: Load intensity data from Excel
intensity_file_path = r"New Average\UV intensities P4.xlsx"
df_intensity = pd.read_excel(intensity_file_path)

# Step 3: Ensure folder names are strings for matching
df_distances["Structure"] = df_distances["Structure"].astype(str)
df_intensity["File Number"] = df_intensity["File Number"].astype(str)

# Step 4: Merge datasets on folder names
merged_df = df_distances.merge(df_intensity, left_on="Structure", right_on="File Number", how="inner")

# Step 5: Extract necessary columns
x_values = merged_df["Smallest_Mg_H_Distance"]  # X-axis: second smallest Mg-H distance
y_values = merged_df["Peak Intensity (Mb)"]  # Y-axis: intensity

# Step 6: Plot the data
plt.figure(figsize=(8, 6))
plt.scatter(x_values, y_values/4, c="blue", edgecolor="black")
plt.title("Smallest Mg-H Distance vs Intensity (P4 + 2H)")
plt.xlabel("Smallest Mg-H Distance (Å)")
plt.ylabel("Intensity (Mb/Si atom)")
plt.grid(True)

for idx, row in merged_df.iterrows():
    plt.annotate(str(row["File Number"]), 
                 (row["Smallest_Mg_H_Distance"], row["Peak Intensity (Mb)"]/4),
                 textcoords="offset points", xytext=(5, 5), fontsize=8)

plt.savefig("Mg_H_distance_example.png", dpi=300)
plt.show()


In [ ]:
#Script producing dataset for Figure 37

import numpy as np
import pandas as pd
from ase import Atoms
from ase.io import read
from pathlib import Path

# Function to calculate bond length between two atoms
def bond_length(atom1, atom2):
    return np.linalg.norm(atom1.position - atom2.position)

# Function to calculate average, smallest, and second smallest Mg-H bond lengths
def mg_h_distances(atoms, element1="Mg", element2="H"):
    bond_lengths = []  # List to store Mg-H bond lengths
    
    for i, atom in enumerate(atoms):
        if atom.symbol == element1:  # Only calculate for Mg atoms
            for j, other_atom in enumerate(atoms):
                if i != j and other_atom.symbol == element2:  # Look for H atoms
                    length = bond_length(atom, other_atom)
                    bond_lengths.append(length)  # Store bond length

    if bond_lengths:
        bond_lengths.sort()  # Sort distances in ascending order
        avg_distance = np.mean(bond_lengths)  # Calculate average bond length
        smallest = bond_lengths[0] if len(bond_lengths) > 0 else None
        second_smallest = bond_lengths[1] if len(bond_lengths) > 1 else None
        return avg_distance, smallest, second_smallest
    else:
        return None, None, None  # Return None if no Mg-H bonds exist

# Step 1: Define the root directory where your subfolders are located
optimisation_root_dir = Path(r"pyroxene_4\final_fhi-aims_data")

# Dictionary to store the results
results = []

# Step 2: Iterate over the folders to find and process each .xyz file
for opt_folder in optimisation_root_dir.iterdir():  # Loop through all folders in the root directory
    if opt_folder.is_dir():  # Ensure it's a directory
        xyz_file = opt_folder / f"final_{opt_folder.name}.xyz"  # Match the final_x.xyz file based on folder name
        
        if xyz_file.exists():
            print(f"Processing {xyz_file}")
            
            # Manually read and process the .xyz file
            with open(xyz_file, "r") as f:
                lines = f.readlines()
                
                # Skip the first line (energy) and use the second line (atom count)
                num_atoms = int(lines[1].strip())
                
                # Extract atomic coordinates starting from the third line
                atom_data = lines[2:2 + num_atoms]
                
                # Convert to ASE Atoms object
                symbols = []
                positions = []
                for line in atom_data:
                    parts = line.split()
                    symbols.append(parts[0])  # First column is element symbol
                    positions.append([float(parts[1]), float(parts[2]), float(parts[3])])  # X, Y, Z coordinates
                
                atoms = Atoms(symbols=symbols, positions=positions)
                
                # Step 4: Calculate Mg-H bond distances
                avg_distance, smallest, second_smallest = mg_h_distances(atoms, element1="Mg", element2="H")
                
                if avg_distance is not None:
                    results.append([opt_folder.name, avg_distance, smallest, second_smallest])
                    print(f"Avg Mg-H: {avg_distance:.2f} Å, Smallest: {smallest:.2f} Å, 2nd Smallest: {second_smallest:.2f} Å")
                else:
                    print(f"No Mg-H bonds found in {opt_folder.name}")
            
            print("\n" + "-"*50)
        else:
            print(f"XYZ file not found in folder {opt_folder.name}")

# Step 5: Convert results to a dataset-like format
results_df = pd.DataFrame(results, columns=["Structure", "Average_Mg_H_Distance", "Smallest_Mg_H_Distance", "Second_Smallest_Mg_H_Distance"])
results_df.to_csv("mg_h_distances_P4.csv", index=False)  # Save as CSV
print("Results saved to 'mg_h_distances_P4.csv'")